# Analysing naming patterns of 'darshinis' in Bangalore

In [53]:
import pandas as pd

In [54]:
df = pd.read_csv("zomato.csv")

In [55]:
#Rename column headers to remove potentially problematic brackets and spaces
df = df.rename(columns = {'approx_cost(for two people)':'cost_for_two'})
df = df.rename(columns = {'listed_in(type)':'type'})
df = df.rename(columns = {'listed_in(city)': 'city'})

In [56]:
udupi = df[df['name'].str.contains('Udupi')]
udupi.shape

(340, 17)

In [57]:
#Repeat with all popular search terms
upahar = df[df['name'].str.contains('Upahar')]
darshini = df[df['name'].str.contains('Darshini')]
bhavan = df[df['name'].str.contains('Bhavan')]
vishnu = df[df['name'].str.contains('Vishnu')]
krishna = df[df['name'].str.contains('Krishna')]
brahmins = df[df['name'].str.contains('Brahmin')]
veg = df[df['name'].str.contains('Veg')]

In [58]:
print("Names with 'Udupi' in it =",len(udupi))
print("Names with 'Upahar' in it =",len(upahar))
print("Names with 'Darshini' in it =",len(darshini))
print("Names with 'Bhavan' in it =",len(bhavan))
print("Names with 'Vishnu' in it =",len(vishnu))
print("Names with 'Krishna' in it =",len(krishna))
print("Names with 'Brahmin' in it =",len(brahmins))
print("Names with 'Veg' in it =",len(veg))

Names with 'Udupi' in it = 340
Names with 'Upahar' in it = 180
Names with 'Darshini' in it = 71
Names with 'Bhavan' in it = 288
Names with 'Vishnu' in it = 57
Names with 'Krishna' in it = 254
Names with 'Brahmin' in it = 27
Names with 'Veg' in it = 545


In [59]:
#Merge all to form sample of darshini names
sample = pd.concat([udupi, upahar, darshini, bhavan, vishnu, krishna, brahmins, veg], keys = ['udupi','upahar', 'darshini', 'bhavan', 'vishnu', 'krishna', 'brahmins', 'veg'])
len(sample)

1762

### Cleaning the sample
1. Remove duplicates
2. Filter out restaurants with > 500 as cost for two - since they don't qualify as darshinis. This requires dropping rows with NaN values for cost
3. Filter out restaurants based on restaurant type (rest_type) and cuisine
4. Drop absolutely unnecessary columns

In [60]:
sample = sample.drop_duplicates()
len(sample)

1585

In [61]:
sample = sample.dropna(subset = ['cost_for_two'])

In [62]:
#filter out restaurants based on cost_for_two
sample['cost_for_two'].describe()

count     1583
unique      17
top        300
freq       375
Name: cost_for_two, dtype: object

In [63]:
sample = sample[(sample['cost_for_two'].astype(int)<500)]

In [64]:
#remove Sweet Shop, Dhaba, Bakery etc.
sample['rest_type'].value_counts()

rest_type
Quick Bites                1082
Casual Dining               153
Takeaway, Delivery           63
Sweet Shop                   40
Sweet Shop, Quick Bites      17
Mess                          6
Delivery                      5
Quick Bites, Sweet Shop       4
Dhaba                         2
Bakery, Quick Bites           2
Takeaway                      1
Name: count, dtype: int64

In [65]:
sample = sample[sample["rest_type"].str.contains("Sweet Shop|Dhaba|Bakery|Mess") == False]
sample['rest_type'].value_counts()

rest_type
Quick Bites           1082
Casual Dining          153
Takeaway, Delivery      63
Delivery                 5
Takeaway                 1
Name: count, dtype: int64

In [66]:
sample['cuisines'].value_counts()

cuisines
South Indian                                        290
South Indian, North Indian, Chinese                 248
South Indian, Chinese                                78
South Indian, North Indian, Chinese, Street Food     66
South Indian, North Indian                           57
                                                   ... 
Biryani                                               1
North Indian, Street Food, Pizza                      1
Italian, Pizza                                        1
South Indian, Fast Food, Juices                       1
North Indian, South Indian, Chinese, Street Food      1
Name: count, Length: 81, dtype: int64

In [67]:
sample = sample[sample['cuisines'].str.contains("South Indian")]

In [68]:
sample = sample.drop('url', axis = 1)

In [69]:

sample = sample.drop('online_order', axis = 1)
sample = sample.drop('book_table', axis = 1)
sample = sample.drop('rate', axis = 1)
sample = sample.drop('votes', axis = 1)
sample = sample.drop('phone', axis = 1)

sample = sample.drop('type', axis = 1)
sample = sample.drop('city', axis = 1)

#### Final sample

In [70]:
sample.shape

(1078, 9)

In [71]:
sample.head()

address  \
udupi 3    1st Floor, Annakuteera, 3rd Stage, Banashankar...   
      149          No.1585, 22nd Main,Banashankari 1st Stage   
      196  31/1, Puttalingaiah Road, Second Stage, Bangal...   
      240  335, KR Road, Shastri Nagar, Yediyur, Jayanaga...   
      244  164/B, 24th Cross, 6th Block Yediyur, Jayanaga...   

                               name      location    rest_type   dish_liked  \
udupi 3       Addhuri Udupi Bhojana  Banashankari  Quick Bites  Masala Dosa   
      149              Udupi Upahar  Banashankari  Quick Bites          NaN   
      196               Udupi Ruchi  Banashankari  Quick Bites          NaN   
      240             Sri Udupi Veg     Jayanagar  Quick Bites          NaN   
      244  New Sri Udupi Restaurant     Jayanagar  Quick Bites          NaN   

                                                    cuisines cost_for_two  \
udupi 3                           South Indian, North Indian          300   
      149        South Indian, Chinese, North Indian, Juices          300   
      196   South Indian, North Indian, Chinese, Street Food          200   
      240                                       South Indian          250   
      244  North Indian, South Indian, Chinese, Beverages...          200   

                                                reviews_list  \
udupi 3    [('Rated 4.0', "RATED\n  Great food and proper...   
      149  [('Rated 1.0', 'RATED\n  Food is not good... N...   
      196  [('Rated 1.0', "RATED\n  Worst option to order...   
      240                                                 []   
      244                                                 []   

                                                   menu_item  
udupi 3                                                   []  
      149  ['Rava Dosa', 'Pepper Masala Dosa', 'Butter Ma...  
      196  ['Idly Vada', 'Onion Dosa', 'Rava Dosa', 'Rava...  
      240                                                 []  
      244                                                 []

In [72]:
sorted(sample.name.unique())

["'Brahmins' Thatte Idli",
 '99 Udupi Veg',
 'A2B - Adyar Ananda Bhavan',
 'A2B Veg - Adyar Ananda Bhavan',
 'Addhuri Udupi Bhojana',
 'Adithya Veg',
 'Adyar Anand Bhavan Sweets',
 'Akshaya Sambrama Veg Restaurant',
 'Amaravati - Pure Veg Restaurant',
 'Annapurna Veg',
 'Ashirvad Veg',
 'Atithi Aramane Veg',
 'Atithi Grand Veg Hotel',
 'BJ Bhavan Veg Restaurants',
 'Bangalore Agarwal Bhavan',
 'Basava Bhavana (B2)',
 'Bhavani Foods',
 'Bhavani Restaurant',
 'Bhavani Upahara',
 'Brahmin Cafe',
 'Brahmin Tiffins & Coffee',
 "Brahmin's Coffee Bar",
 'Brahmins Food Corner',
 "Brahmins' Thatte Idli",
 'Brindavan Veg',
 'Cafe Udupi Ruchi',
 'Chaitanya Veg',
 'Chaithanya Veg',
 'Chamarajpet Non Veg Restaurant',
 'Chandra Prakesh Veg Restaurant',
 'CraveBelly Veg & Non veg',
 "Dambal's Veg",
 'Ganesh Bhavan',
 'Ganesh Darshini',
 'Gangothri Veg Restaurant',
 'HSR Agarval Bhavan',
 'HVR Veg',
 'Hare Krishna',
 'Hare Krishna Kuteera',
 'Hotel Ayodhya Upahar',
 'Hotel Kadamba Veg',
 'Hotel Mahesh

In [73]:
sample.cost_for_two.median()

300.0

In [74]:
len(sample)

1078

In [75]:
sample.location.value_counts()

location
BTM                           117
Jayanagar                     100
Shivajinagar                   66
HSR                            66
Basavanagudi                   37
                             ... 
ITPL Main Road, Whitefield      2
Sanjay Nagar                    1
Sadashiv Nagar                  1
Seshadripuram                   1
Hebbal                          1
Name: count, Length: 62, dtype: int64

In [78]:
sample['name'].str.contains("Sri|Shree|Shri").value_counts()

name
False    815
True     263
Name: count, dtype: int64

In [ ]:
sample.to_excel("darshini_sample.xlsx")

In [ ]:
pd.set_option('display.max_rows', None)
sample.name.value_counts()

name
A2B - Adyar Ananda Bhavan                   49
Sri Udupi Park                              34
Sri Krishna Sagar                           30
New Udupi Grand                             25
Vijayalakshmi Veg                           23
New Udupi Upahar                            21
Sree Krishna Kafe                           17
Upahara Darshini                            14
Indira Darshini                             14
Udupi Grand                                 13
Vegetarea                                   13
Krishna Vaibhava                            12
Shilpa Vegetarian Restaurant                12
Shilpa Veg                                  12
Chaitanya Veg                               12
Udupi Ruchi                                 12
Sri Ayodhya Veg                             11
Sri Udupi Grand                             11
Swaadam Veg                                 11
Sri Saravana Bhavan                         11
Krishna Sagar                               11
Upahara 

In [ ]:
names = sample[['name']]
names = names.drop_duplicates()

NameError: name 'sample' is not defined

In [ ]:
names_split = names['name'].str.split(' ', expand = True)
names_split

0              1                 2            3  \
udupi    3             Addhuri          Udupi           Bhojana         None   
         149             Udupi         Upahar              None         None   
         196             Udupi          Ruchi              None         None   
         240               Sri          Udupi               Veg         None   
         244               New            Sri             Udupi   Restaurant   
         297             Udupi          Ruchi             Grand         None   
         342             Udupi          Grand              None         None   
         737             Shree          Udupi             South      kitchen   
         1044            Shree          Udupi             Grand         None   
         1173              New          Udupi             Grand         None   
         1330            Udupi        Upachar              None         None   
         1388            Udupi          Thaja            Thindi         None   
         1618            Udupi       Biriyani             Point         None   
         1750            Udupi         Thadka              None         None   
         1790            Udupi           Park               Inn         None   
         2373            Udupi           Cafe              None         None   
         2841              New          Udupi            Upahar         None   
         3385              New          Udupi             Anand       Bhavan   
         3842            Udupi       Aatithya              None         None   
         3905            Udupi       Nalapaka              None         None   
         3977            Udupi           Savi            Thindi         None   
         5368              Sri          Udupi              Park         None   
         6032              Sri          Udupi             Grand         None   
         6874            Udupi          Grand               Veg         None   
         6969            Udupi          Delux              None         None   
         7222            Udupi            Veg           Express         None   
         7251               99          Udupi               Veg         None   
         7329              New          Udupi          Delicacy         None   
         7586              New          Udupi             Shubh        Sagar   
         9633              New          Udupi         Dharshini         None   
         10634           Udupi          Gokul              Cafe         None   
         10772             Sri          Udupi             Sagar         None   
         13458             New          Udupi            Palace         None   
         13589           Udupi           Food          Junction         None   
         14136           Urban          Udupi               Veg   Restaurant   
         16009           Udupi         Bhavan              None         None   
         16018           Udupi         Palace              None         None   
         17457           Udupi        Upahara              None         None   
         18020             Sri          Udupi              None         None   
         19158             New          Udupi            Garden         None   
         23982           Udupi            Sri           Krishna         Cafe   
         24303           Udupi       Gardenia              None         None   
         24799             New          Udupi           Upahara         None   
         25054             New          Udupi              Park         None   
         25112             New          Udupi              Park       Deluxe   
         39934           Udupi         Garden              None         None   
         40697           Udupi       Brahmins              Cafe         None   
         41308             Sri          Udupi            Deluxe         None   
         41602           Udupi        Vaibhav              None         None   
         42050           Udup

In [ ]:
names_split = names_split.rename(columns = {'O':'prefix'})
names_split

0              1                 2            3  \
udupi    3             Addhuri          Udupi           Bhojana         None   
         149             Udupi         Upahar              None         None   
         196             Udupi          Ruchi              None         None   
         240               Sri          Udupi               Veg         None   
         244               New            Sri             Udupi   Restaurant   
         297             Udupi          Ruchi             Grand         None   
         342             Udupi          Grand              None         None   
         737             Shree          Udupi             South      kitchen   
         1044            Shree          Udupi             Grand         None   
         1173              New          Udupi             Grand         None   
         1330            Udupi        Upachar              None         None   
         1388            Udupi          Thaja            Thindi         None   
         1618            Udupi       Biriyani             Point         None   
         1750            Udupi         Thadka              None         None   
         1790            Udupi           Park               Inn         None   
         2373            Udupi           Cafe              None         None   
         2841              New          Udupi            Upahar         None   
         3385              New          Udupi             Anand       Bhavan   
         3842            Udupi       Aatithya              None         None   
         3905            Udupi       Nalapaka              None         None   
         3977            Udupi           Savi            Thindi         None   
         5368              Sri          Udupi              Park         None   
         6032              Sri          Udupi             Grand         None   
         6874            Udupi          Grand               Veg         None   
         6969            Udupi          Delux              None         None   
         7222            Udupi            Veg           Express         None   
         7251               99          Udupi               Veg         None   
         7329              New          Udupi          Delicacy         None   
         7586              New          Udupi             Shubh        Sagar   
         9633              New          Udupi         Dharshini         None   
         10634           Udupi          Gokul              Cafe         None   
         10772             Sri          Udupi             Sagar         None   
         13458             New          Udupi            Palace         None   
         13589           Udupi           Food          Junction         None   
         14136           Urban          Udupi               Veg   Restaurant   
         16009           Udupi         Bhavan              None         None   
         16018           Udupi         Palace              None         None   
         17457           Udupi        Upahara              None         None   
         18020             Sri          Udupi              None         None   
         19158             New          Udupi            Garden         None   
         23982           Udupi            Sri           Krishna         Cafe   
         24303           Udupi       Gardenia              None         None   
         24799             New          Udupi           Upahara         None   
         25054             New          Udupi              Park         None   
         25112             New          Udupi              Park       Deluxe   
         39934           Udupi         Garden              None         None   
         40697           Udupi       Brahmins              Cafe         None   
         41308             Sri          Udupi            Deluxe         None   
         41602           Udupi        Vaibhav              None         None   
         42050           Udup

In [ ]:
names_split.to_excel("names.xlsx")

In [ ]:
df = pd.read_csv("names_clean.csv")
df

,prefix,suffix_1,suffix_2,suffix_3,suffix_4,suffix_5,suffix_6
0,Parisar,Veg,Restaurant,NaN,NaN,NaN,NaN
1,Vakulaa,Veg,Restaurant,NaN,NaN,NaN,NaN
2,Vishwamitra,Veg,Restaurant,NaN,NaN,NaN,NaN
3,Skandas,Veg,Restaurant,NaN,NaN,NaN,NaN
4,Gangothri,Veg,Restaurant,NaN,NaN,NaN,NaN
5,Loukya,Veg,Restaurant,NaN,NaN,NaN,NaN
6,Sitara,Veg,Restaurant,NaN,NaN,NaN,NaN
7,Udupas,Veg,Restaurant,NaN,NaN,NaN,NaN
8,Mayur,Veg,Restaurant,NaN,NaN,NaN,NaN
9,Pavana,Veg,Restaurant,NaN,NaN,NaN,NaN


In [ ]:
df.prefix.value_counts().head()

prefix
Sri        49
Udupi      29
New        18
Hotel      10
Upahara     6
Name: count, dtype: int64

In [ ]:
df.suffix_1.value_counts().head()

suffix_1
Veg        42
Udupi      28
Upahara    20
Krishna    17
Bhavan     10
Name: count, dtype: int64

In [ ]:
df.suffix_2.value_counts().head()

suffix_2
Veg           26
Bhavan        14
Restaurant    11
Grand         10
Upahara       10
Name: count, dtype: int64

In [ ]:
df.suffix_3.value_counts().head()

suffix_3
Veg           8
Restaurant    6
Bhavan        5
Hotel         3
&             2
Name: count, dtype: int64

In [ ]:
df.suffix_4.value_counts().head()

suffix_4
&             3
Non-Veg       3
Restaurant    2
Veg           2
Bhavan        1
Name: count, dtype: int64